In [ ]:
%cd ..

import pandas as pd
from bankroll import *
from bankroll.brokers import ibkr
from ib_insync import IB, util
from random import randint

In [ ]:
# See bankroll's README for more information on setting up these parameters.

# The port upon which Interactive Brokers' Trader Workstation (or IB Gateway) is accepting connections.
TWS_PORT = 4001

# Token ID from IB's Flex Web Service:
# https://www.interactivebrokers.com/en/software/am/am/reports/flex_web_service_version_3.htm
FLEX_TOKEN = ''

# Query ID for Trades report from IB's Flex Web Service:
# https://www.interactivebrokers.com/en/software/am/am/reports/flex_web_service_version_3.htm
FLEX_QUERY_TRADES = 0

# Query ID for Activity report from IB's Flex Web Service:
# https://www.interactivebrokers.com/en/software/am/am/reports/flex_web_service_version_3.htm
FLEX_QUERY_ACTIVITY = 0

In [ ]:
util.startLoop()
ib = IB()
ib.connect('127.0.0.1', port = TWS_PORT, clientId = randint(1, 100000000))

In [ ]:
positions = ibkr.downloadPositions(ib, lenient=False)

In [ ]:
stockPositions = [p for p in positions if isinstance(p.instrument, Stock)]
stockPositions.sort(key=lambda p: p.instrument)
stockPositions

In [ ]:
activity = ibkr.downloadTrades(token=FLEX_TOKEN,
                               queryID=FLEX_QUERY_TRADES,
                               lenient=False)
activity += ibkr.downloadNonTradeActivity(token=FLEX_TOKEN,
                                          queryID=FLEX_QUERY_ACTIVITY,
                                          lenient=False)
len(activity)

In [ ]:
values = liveValuesForPositions(positions, ibkr.IBDataProvider(ib))

In [ ]:
realizedBases = {p: realizedBasisForSymbol(p.instrument.symbol, activity) for p in stockPositions}

In [ ]:
rows = {p.instrument.symbol: [
    values[p],
    p.costBasis,
    realizedBases[p],
] for p in stockPositions}

In [ ]:
df = pd.DataFrame.from_dict(data=rows, orient='index', columns=[
    'Market value',
    'Cost basis',
    'Realized basis',
])

df